In [5]:
import numpy as np
import pandas as pd
import scipy.stats as st

In [3]:
wine_df = pd.read_csv('winequality-red.csv')

# **Критерий КОЛМОГОРОВА-СМИРНОВА**

**запишем функцию для вычисления**

In [94]:
def kolmogorov(data, loc, scale):
    D_plus = np.zeros(data.shape[0])
    D_minus = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        D_plus[i] = (i / data.shape[0]) - st.norm.cdf(data[i], loc=loc, scale=scale)
        D_minus[i] = st.norm.cdf(data[i], loc=loc, scale=scale) - ((i - 1) / data.shape[0])
    D = np.max([np.max(D_plus), np.max(D_minus)])
    return (6 * data.shape[0] * D + 1) / (6 * np.sqrt(data.shape[0]))

**вычислим критерий соответствия данных wine_df['pH'] с нормальным распределением**

**Для начала определим матожидание. Применим 95% доверительный интервал. Берем Z статистику, поскольку у нас много значений в выборке**

In [95]:
Cl = wine_df['pH'].mean() - 1.96 * np.std(wine_df['pH'], ddof=1) / np.sqrt(wine_df['pH'].shape[0])
Cr = wine_df['pH'].mean() + 1.96 * np.std(wine_df['pH'], ddof=1) / np.sqrt(wine_df['pH'].shape[0])
print(Cl, Cr)
print(f"прикидочное среднее: {np.mean(np.array([Cl, Cr]))}")

3.303545893815603 3.318680497679083
прикидочное среднее: 3.311113195747343


**черт с ним, будем брать среднее и дисперсию выборкию Посчитаем Колмогорова**

In [96]:
kolmogorov(wine_df['pH'], loc=wine_df['pH'].mean(), scale=np.std(wine_df['pH'], ddof=1))

38.888592320993155

**Че то много. Протестируем Колмогорова на заведомо нормально распределенной выборке**

In [97]:
test_data = st.norm.rvs(loc=5, scale=7, size=2000)
kolmogorov(test_data, loc=test_data.mean(), scale=np.std(test_data, ddof=1))

43.263634631753334

**??????????????????????????????????????????????????????????????????????????????????????????????????????**

**попробуем воспользоваться scipy**

In [98]:
st.kstest(wine_df['pH'], st.norm.cdf, args=(wine_df['pH'].mean(), np.std(wine_df['pH'], ddof=1)))

KstestResult(statistic=0.040368454197074355, pvalue=0.010604884802440944)

In [99]:
st.kstest(test_data, st.norm.cdf, args=(5, 7))

KstestResult(statistic=0.019231096131231085, pvalue=0.44468431674920494)

# **Критерий Омега-квадрат**

**напишем функцию вычисления статистики омега квадрат**

In [123]:
def omega_square(data, loc, scale):
    return (1 / 12 / data.shape[0]) + np.sum([(st.norm.cdf(data[i], loc=loc, scale=scale) - (2 * i) / (2 * data.shape[0])) ** 2 for i in range(data.shape[0])])

**вычислим статистику омега квадрат для столбца density**

In [124]:
omega_square(wine_df['density'], np.mean(wine_df['density']), np.std(wine_df['density'], ddof=1))

358.9006714768003

In [125]:
test_data = st.norm.rvs(loc=5, scale=7, size=2000)
omega_square(test_data, test_data.mean(), np.std(test_data, ddof=1))

342.6606588029061

**????????????????????????????????**